## 2장. 트랜스포머 
#### 도전 과제 1번. 요약 하기

In [1]:
from transformers import pipeline

2025-09-21 22:44:59.169379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758462299.186277 1326337 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758462299.191211 1326337 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758462299.208810 1326337 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758462299.208821 1326337 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758462299.208824 1326337 computation_placer.cc:177] computation placer alr

In [2]:
long_text = """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""

In [3]:
# 1) 요약 전용 파이프라인 
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device_map="auto")

sum_supervised = summarizer(
    long_text,
    max_length=150,  
    min_length=60,  
    do_sample=False, 
)[0]["summary_text"]

print("=== [A] Summarization Model (supervised) ===\n", sum_supervised, "\n")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


=== [A] Summarization Model (supervised) ===
 America has changed dramatically during recent years. The number of engineering graduates in the U.S. has declined. China and India graduate six and eight times as many traditional engineers as does the United States. Other industrial countries at minimum maintain their output, while America suffers an increasingly serious decline in the number of engineers. 



In [4]:
# 2) 제로샷
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B", device_map="auto")

zero_shot_prompt = (
    "Summarize the following text in 2–3 sentences, focusing on the main claims and comparisons.\n\n"
    f"Text:\n{long_text.strip()}\n\nSummary:"
)

sum_zero_shot = generator(
    zero_shot_prompt,
    max_new_tokens=120,
    temperature=0.3,    
    top_p=0.9,
    do_sample=True,
    return_full_text=False,
)[0]["generated_text"].strip()

print("=== [B] Zero-shot (instruction only) ===\n", sum_zero_shot, "\n")

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== [B] Zero-shot (instruction only) ===
 America has changed dramatically during recent years. Not only has the number
    of graduates in traditional engineering disciplines such as mechanical,
    civil, electrical, chemical, and aeronautical engineering declined, but in
    most of the premier American universities engineering curricula now concentrate
    on and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely 



In [5]:
# 3) 퓨샷 프롬프트
few_shot_prompt = """You are a concise academic summarizer.
Write a 2–3 sentence summary that preserves key facts and avoids speculation.

Example 1
Text: "The company reduced costs by 30% in 2023 while expanding into two new markets. However, supply-chain issues delayed three product launches."
Summary: "In 2023 the company cut costs by 30% and entered two new markets, but supply-chain problems delayed three launches."

Example 2
Text: "A study of 2,000 participants found that daily walking correlated with improved cardiovascular markers. The authors noted limitations in self-reported activity."
Summary: "A 2,000-person study links daily walking to better cardiovascular health, though reliance on self-reports limits certainty."

Now summarize the following text in 2–3 sentences.

Text: """ + long_text.strip() + "\nSummary:"

sum_few_shot = generator(
    few_shot_prompt,
    max_new_tokens=120,
    temperature=0.2,   # 더 보수적으로
    top_p=0.9,
    do_sample=True,
    return_full_text=False,
)[0]["generated_text"].strip()

print("=== [C] Few-shot (with examples) ===\n", sum_few_shot, "\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== [C] Few-shot (with examples) ===
 "America has changed dramatically during recent years. Not only has
    the number of graduates in traditional engineering disciplines such as
    mechanical, civil, electrical, chemical, and aeronautical engineering declined,
    but in most of the premier American universities engineering curricula now
    concentrate on and encourage largely the study of engineering science. As a
    result, there are declining offerings in engineering subjects dealing with
    infrastructure, the environment, and related issues, and greater concentration
    on high technology subjects, largely supporting 

